In [2]:
# Datensätze importieren, umformen und zusammenführen

import pandas as pd
from scipy import stats

# Lade die CSV-Datei aus deinem lokalen Verzeichnis
wetter = pd.read_csv('wetter.csv')
umsatz = pd.read_csv('umsatzdaten_gekuerzt.csv')
kiwo = pd.read_csv('kiwo.csv')

# Umwandlung der 'Datum' Spalte in das Datumsformat
wetter['Datum'] = pd.to_datetime(wetter['Datum'])
umsatz['Datum'] = pd.to_datetime(umsatz['Datum'])
kiwo['Datum'] = pd.to_datetime(kiwo['Datum'])

# Den Monat aus der 'Datum' Spalte extrahieren
wetter['Monat'] = wetter['Datum'].dt.month

# Füge die Wochentag-Spalte hinzu
umsatz['Wochentag'] = umsatz['Datum'].dt.day_name()

# Definiere die Wochentagsreihenfolge
order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Zuordnung von Warengruppen zu Kategorien
kategorie_mapping = {
    1: 'Brot',
    2: 'Broetchen',
    3: 'Croissant',
    4: 'Konditorei',
    5: 'Kuchen',
    6: 'Saisonbrot'
}

# Neue Spalte 'Kategorie' hinzufügen, basierend auf der Spalte 'Warengruppe'
umsatz['Kategorie'] = umsatz['Warengruppe'].map(kategorie_mapping)

# Schrittweise Zusammenführen der DataFrames mit einem Outer Join
umsatz_gesamt = pd.merge(umsatz, kiwo, on='Datum', how='outer')
umsatz_gesamt = pd.merge(umsatz_gesamt, wetter, on='Datum', how='outer')

# Sortieren nach Datum 
umsatz_gesamt = umsatz_gesamt.sort_values(by='Datum')

#Spalte "KielerWoche" umformen, sodass "1" oder "0" angezeigt wird
umsatz_gesamt['KielerWoche'] = umsatz_gesamt['KielerWoche'].apply(
    lambda x: '1' if x == 1.0 else '0'
)

# Zeilen mit NaN-Werten in der Umsatz-Spalte löschen
umsatz_gesamt = umsatz_gesamt.dropna(subset=['Umsatz'])

# Ausgabe des zusammengeführten DataFrames
print(umsatz_gesamt)

          Datum  Warengruppe      Umsatz Wochentag   Kategorie KielerWoche  \
398  2013-07-01          5.0  317.475875    Monday      Kuchen           0   
397  2013-07-01          4.0   65.890169    Monday  Konditorei           0   
395  2013-07-01          2.0  535.856285    Monday   Broetchen           0   
396  2013-07-01          3.0  201.198426    Monday   Croissant           0   
394  2013-07-01          1.0  148.828353    Monday        Brot           0   
...         ...          ...         ...       ...         ...         ...   
9765 2018-07-31          5.0  261.341313   Tuesday      Kuchen           0   
9764 2018-07-31          4.0   57.102795   Tuesday  Konditorei           0   
9762 2018-07-31          2.0  586.081666   Tuesday   Broetchen           0   
9761 2018-07-31          1.0  123.075682   Tuesday        Brot           0   
9763 2018-07-31          3.0  285.872616   Tuesday   Croissant           0   

      Bewoelkung  Temperatur  Windgeschwindigkeit  Wettercode  

In [3]:
# Hausaufgabe Tag 1: Durchschnittstemperaturen bestimmen

# Filtern der Daten für Juni (Monat == 6)
wetter_juni = wetter[wetter['Monat'] == 6]

# Berechnung Gesamtdurchschnittstemperatur
durchschnittstemperatur = wetter['Temperatur'].mean()
print(f"Die Gesamtdurchschnittstemperatur beträgt: {durchschnittstemperatur:.2f} °C")

# Durchschnittstemperatur im Juni berechnen
durchschnittstemperatur_juni = wetter_juni['Temperatur'].mean()

# Ausgabe des Ergebnisses
print(f"Die Durchschnittstemperatur im Juni beträgt: {durchschnittstemperatur_juni:.2f} °C")

# Den Monat aus der 'Datum' Spalte extrahieren, falls noch nicht geschehen
wetter['Datum'] = pd.to_datetime(wetter['Datum'])
wetter['Monat'] = wetter['Datum'].dt.month

# Filtern der Daten für Mai (Monat 5) und Juli (Monat 7)
wetter_mai = wetter[wetter['Monat'] == 5]
wetter_juli = wetter[wetter['Monat'] == 7]

# Berechnung der Durchschnittstemperaturen für Mai und Juli
durchschnitt_mai = wetter_mai['Temperatur'].mean()
durchschnitt_juli = wetter_juli['Temperatur'].mean()

# Ausgabe der Durchschnittstemperaturen
print(f"Durchschnittstemperatur im Mai: {durchschnitt_mai:.2f} °C")
print(f"Durchschnittstemperatur im Juli: {durchschnitt_juli:.2f} °C")

# Durchführung des t-Tests, um zu überprüfen, ob der Unterschied signifikant ist
# Annahme: Wir testen auf einen zweiseitigen Test (ob die Temperaturen sich signifikant unterscheiden)

t_stat, p_value = stats.ttest_ind(wetter_mai['Temperatur'].dropna(), wetter_juli['Temperatur'].dropna())

# Ausgabe des t-Test-Ergebnisses
print(f"t-Statistik: {t_stat:.3f}")
print(f"p-Wert: {p_value:.3f}")

# Interpretation des p-Werts
alpha = 0.05  # Signifikanzniveau
if p_value < alpha:
    print("Der Unterschied in den Durchschnittstemperaturen zwischen Mai und Juli ist signifikant.")
else:
    print("Der Unterschied in den Durchschnittstemperaturen zwischen Mai und Juli ist nicht signifikant.")


Die Gesamtdurchschnittstemperatur beträgt: 12.10 °C
Die Durchschnittstemperatur im Juni beträgt: 18.48 °C
Durchschnittstemperatur im Mai: 15.24 °C
Durchschnittstemperatur im Juli: 20.75 °C
t-Statistik: -15.937
p-Wert: 0.000
Der Unterschied in den Durchschnittstemperaturen zwischen Mai und Juli ist signifikant.


In [ ]:
# Visualisierung der Daten: Umsatz pro Wochentag mit Konfidenzintervall

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

# Konvertiere die 'Datum'-Spalte in das Datumsformat
umsatz['Datum'] = pd.to_datetime(umsatz['Datum'])

# Füge die Wochentag-Spalte hinzu
umsatz['Wochentag'] = umsatz['Datum'].dt.day_name()

# Definiere die Wochentagsreihenfolge
order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Berechne den durchschnittlichen Umsatz und das Konfidenzintervall pro Wochentag
umsatz_summary = umsatz.groupby('Wochentag')['Umsatz'].agg(['mean', 'std', 'count']).reindex(order)
umsatz_summary['sem'] = umsatz_summary['std'] / np.sqrt(umsatz_summary['count'])  # Standardfehler
umsatz_summary['ci'] = umsatz_summary['sem'] * stats.t.ppf((1 + 0.95) / 2, umsatz_summary['count'] - 1)  # 95% Konfidenzintervall

# Erstelle das Balkendiagramm mit einheitlicher Farbe
plt.figure(figsize=(10, 6))
sns.barplot(
    x=umsatz_summary.index,
    y=umsatz_summary['mean'],
    yerr=umsatz_summary['ci'],
    order=order,
    color='skyblue'  # Einheitliche Farbe für alle Balken
)
plt.xlabel('Wochentag')
plt.ylabel('Durchschnittlicher Umsatz')
plt.title('Durchschnittlicher Umsatz pro Wochentag mit Konfidenzintervallen')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
#Darstellung der Umsatzdaten in Abhängigkeit zur Kieler Woche, 
#Lösungsvorschlag von Moritz

import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np



# Merging data frames

#Zuordnung von Warengruppen zu Kategorien
kategorie_mapping = {
    1: 'Brot',
    2: 'Broetchen',
    3: 'Croissant',
    4: 'Konditorei',
    5: 'Kuchen',
    6: 'Saisonbrot'
}
#Neue Spalte 'Kategorie' hinzufügen, basierend auf der Spalte 'Warengruppe'
umsatz['Kategorie'] = umsatz['Warengruppe'].map(kategorie_mapping)
#Überprüfen, ob die Zuordnung korrekt durchgeführt wurde
print(umsatz.head())

# Convert the 'Datum' column to datetime format to work with dates
umsatz['Datum'] = pd.to_datetime(umsatz['Datum'])

# Extract the weekday name and add it as a new column
umsatz['Weekday'] = umsatz['Datum'].dt.day_name()

# Calculate the mean and standard error for each weekday
weekday_stats = umsatz.groupby('Weekday')['Umsatz'].agg(['mean', 'count', 'std'])

# Calculate the 95% confidence interval
confidence_level = 0.95
z_score = stats.norm.ppf((1 + confidence_level) / 2)  # z-score for 95% confidence
weekday_stats['ci'] = z_score * (weekday_stats['std'] / np.sqrt(weekday_stats['count']))

# Sort weekdays in calendar order
ordered_weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
weekday_stats = weekday_stats.reindex(ordered_weekdays)

# Convert the 'Datum' column to datetime for consistency
kiwo['Datum'] = pd.to_datetime(kiwo['Datum'])

# Merge Kieler Woche data with main DataFrame
umsatz = umsatz.merge(kiwo, on='Datum', how='left')

# Add the 'Kieler Woche' column
umsatz['Kieler Woche'] = umsatz['KielerWoche'].apply(lambda x: 'yes' if x == 1 else 'no')

# Drop the temporary 'KielerWoche' column
umsatz.drop(columns=['KielerWoche'], inplace=True)

# Check the updated DataFrame
print(umsatz.head())



# Average Umsatz by weekday

# Plotting with confidence intervals
plt.figure(figsize=(10, 6))
plt.bar(weekday_stats.index, weekday_stats['mean'], yerr=weekday_stats['ci'], 
        color='skyblue', edgecolor='black', capsize=5, alpha=0.7)

# Add labels and title
plt.title('Average Umsatz by Weekday with 95% Confidence Intervals')
plt.xlabel('Weekday')
plt.ylabel('Average Umsatz')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



# Average Umsatz during and outside Kieler Woche

# Calculate the mean, count, and standard deviation for Umsatz grouped by 'Kieler Woche'
kieler_stats = umsatz.groupby('Kieler Woche')['Umsatz'].agg(['mean', 'count', 'std'])

# Calculate the 95% confidence interval
kieler_stats['ci'] = z_score * (kieler_stats['std'] / np.sqrt(kieler_stats['count']))

# Plotting with confidence intervals
plt.figure(figsize=(8, 5))
plt.bar(
    kieler_stats.index, kieler_stats['mean'], yerr=kieler_stats['ci'], 
    color=['skyblue', 'salmon'], edgecolor='black', capsize=5, alpha=0.7
)

# Add labels and title
plt.title('Average Umsatz During and Outside Kieler Woche with 95% Confidence Intervals')
plt.xlabel('Kieler Woche')
plt.ylabel('Average Umsatz')
plt.xticks(ticks=[0, 1], labels=['No', 'Yes'], rotation=0)
plt.tight_layout()
plt.show()



# Average Umsatz by weekday during and outside Kieler Woche

# Group by Weekday and 'Kieler Woche' to calculate mean, count, and std
weekday_kieler_stats = (
    umsatz.groupby(['Weekday', 'Kieler Woche'])['Umsatz']
    .agg(['mean', 'count', 'std'])
    .reset_index()
)

# Calculate confidence intervals
weekday_kieler_stats['ci'] = (
    z_score * (weekday_kieler_stats['std'] / np.sqrt(weekday_kieler_stats['count']))
)

# Pivot the data for easier plotting
weekday_kieler_pivot = weekday_kieler_stats.pivot(
    index='Weekday', columns='Kieler Woche', values=['mean', 'ci']
)

# Ensure the weekdays are in calendar order
weekday_kieler_pivot = weekday_kieler_pivot.loc[ordered_weekdays]

# Plotting
plt.figure(figsize=(12, 6))
x = np.arange(len(ordered_weekdays))  # the label locations
width = 0.35  # the width of the bars

# Bars for "No" (Kieler Woche == 'no')
plt.bar(
    x - width/2, 
    weekday_kieler_pivot[('mean', 'no')], 
    yerr=weekday_kieler_pivot[('ci', 'no')], 
    width=width, 
    label='No', 
    color='skyblue', 
    capsize=5, 
    edgecolor='black', 
    alpha=0.7
)

# Bars for "Yes" (Kieler Woche == 'yes')
plt.bar(
    x + width/2, 
    weekday_kieler_pivot[('mean', 'yes')], 
    yerr=weekday_kieler_pivot[('ci', 'yes')], 
    width=width, 
    label='Yes', 
    color='salmon', 
    capsize=5, 
    edgecolor='black', 
    alpha=0.7
)

# Add labels, title, and legend
plt.title('Average Umsatz by Weekday During and Outside Kieler Woche with 95% Confidence Intervals')
plt.xlabel('Weekday')
plt.ylabel('Average Umsatz')
plt.xticks(x, ordered_weekdays)
plt.legend(title='Kieler Woche')
plt.tight_layout()
plt.show()



# Average Umsatz by Kategorie

# Calculate the mean, count, and standard deviation for Umsatz grouped by Kategorie
category_stats = umsatz.groupby('Kategorie')['Umsatz'].agg(['mean', 'count', 'std'])

# Calculate the 95% confidence intervals
category_stats['ci'] = z_score * (category_stats['std'] / np.sqrt(category_stats['count']))

# Plotting with confidence intervals
plt.figure(figsize=(10, 6))
plt.bar(
    category_stats.index, category_stats['mean'], yerr=category_stats['ci'], 
    color='lightgreen', edgecolor='black', capsize=5, alpha=0.8
)

# Add labels and title
plt.title('Average Umsatz by Kategorie with 95% Confidence Intervals')
plt.xlabel('Kategorie')
plt.ylabel('Average Umsatz')
plt.tight_layout()
plt.show()



# Average Umsatz by Kategorie for each weekday

# Calculate statistics grouped by Kategorie and Weekday
category_weekday_stats = (
    umsatz.groupby(['Kategorie', 'Weekday'])['Umsatz']
    .agg(['mean', 'count', 'std'])
    .reset_index()
)

# Calculate the 95% confidence intervals
category_weekday_stats['ci'] = z_score * (category_weekday_stats['std'] / np.sqrt(category_weekday_stats['count']))

# Sort weekdays in calendar order
category_weekday_stats['Weekday'] = pd.Categorical(category_weekday_stats['Weekday'], categories=ordered_weekdays, ordered=True)
category_weekday_stats.sort_values(['Kategorie', 'Weekday'], inplace=True)

# Plotting
plt.figure(figsize=(14, 8))
categories = category_weekday_stats['Kategorie'].unique()
x = np.arange(len(categories))  # label locations
width = 0.1  # width of the bars

# Loop through weekdays to plot separate bars
for i, weekday in enumerate(ordered_weekdays):
    weekday_data = category_weekday_stats[category_weekday_stats['Weekday'] == weekday]
    plt.bar(
        x + i * width, weekday_data['mean'], width, 
        yerr=weekday_data['ci'], label=weekday, capsize=5, alpha=0.8
    )

# Add labels, title, and legend
plt.title('Average Umsatz by Kategorie for Each Weekday with 95% Confidence Intervals')
plt.xlabel('Kategorie')
plt.ylabel('Average Umsatz')
plt.xticks(ticks=x + (width * 3), labels=categories)
plt.legend(title='Weekday', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()